In [1]:
import torch
import pytorch_lightning as pl
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt

from torch import optim
from torch import nn

from ch09_random_walk import RandomWalk

In [2]:
class redNeuronal(nn.Module):
    def __init__(self, numero_de_estados_agregados):
        super().__init__()
        self.fc1 = nn.Linear(numero_de_estados_agregados, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 1)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [3]:
np.random.seed(666)
torch.manual_seed(666)


numero_de_estados_agregados = 100
estados_agregados = np.linspace(0, 1001, numero_de_estados_agregados)

In [4]:
mdp = RandomWalk()
model = redNeuronal(numero_de_estados_agregados+1)
optimizer = optim.SGD(model.parameters(), lr=0.0001)
criterio = nn.MSELoss()

In [5]:
recompensas = []
error_acumulado = []

episodios_realizados = 8000
epochs = 100


for j in range(0, epochs):
    error_total = 0
    estados_visitados = []
    recompensas = []
    for i in range(0,episodios_realizados):
        mdp.reset_state()
        while True:
            estado, recompensa, terminal = mdp.step()
            if terminal:
                episodio_agregado = np.digitize(mdp.states_visited, estados_agregados)
                episodio_agregado = list(set(episodio_agregado))
                estados_visitados.append(episodio_agregado)
                recompensas.append(recompensa)
                break
                
    estados_visitados_matriz = sp.lil_matrix((episodios_realizados, numero_de_estados_agregados+1), dtype=int)
    for row_index, col_indices in enumerate(estados_visitados):
        estados_visitados_matriz[row_index, col_indices] = 1
    data_para_entrenar = estados_visitados_matriz.toarray()
    tensor_data_para_entrenar = torch.from_numpy(data_para_entrenar).float()
    y = torch.FloatTensor(recompensas).view(episodios_realizados, -1)

    optimizer.zero_grad()
    pred = model.forward(tensor_data_para_entrenar)
    error_parcial = criterio(pred, y)
    error_parcial.backward()
    error_total += error_parcial.item()
    error_acumulado.append(error_parcial.item())
    optimizer.step()
    if j % 10 == 0:
        print(error_total)

0.5325416922569275
0.5266414284706116


KeyboardInterrupt: 

In [ ]:
estados_agredados_valor = torch.from_numpy(np.identity(numero_de_estados_agregados+1)).float()
valor_estados = model.forward(estados_agredados_valor)
valores_finales = [l[0] for l in valor_estados.tolist()]

In [ ]:
plt.plot(valores_finales)

In [ ]:
plt.plot(error_acumulado, color='r')